# Modelagem de classificação de clientes com dados da saúde suplementar

Vamos usar os [dados obtidos anteriormente](https://github.com/VFLins/hospitalar_data_ans/blob/main/notebooks/collect_routine_ans.ipynb) e o conhecimento que obtivemos na sua [análise exploratória](https://vflins.github.io/hospitalar_data_ans/pages/eda.html) para produzir uma solução de dados com valor de negócio para empresas do **setor de operação de saúde.**

A proposta deste modelo é produzir um indicador do *"custo potencial de aquisição"* de um cliente. Esta solução pode facilitar grandemente a precificação dos produtos de saúde, além disso, neste processo podemos acabar aprendendo mais sobre os diferentes tipos de perfis de clientes e suas necessidades!

In [1]:
# imports

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

24/04/07 17:18:16 WARN Utils: Your hostname, DESKTOP-RFN0UB8 resolves to a loopback address: 127.0.1.1; using 172.18.59.98 instead (on interface eth0)
24/04/07 17:18:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/07 17:18:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# variaveis de conexao

drivername = "oracle.jdbc.OracleDriver"
WALLET_PLACE = "/home/user/Downloads/wallet_demodb"
URL = f"jdbc:oracle:thin:@demodb_medium?TNS_ADMIN={WALLET_PLACE}"
USR = "ADMIN"
PWD = getpass("Insira a senha de administrador do banco de dados: ")

In [ ]:
# conjuntos de dados

tablename = "HOSP_CONS"
query = f"SELECT * FROM {tablename}"

cons_df = spark.read.format("jdbc")\
  .option("driver", drivername)\
  .option("query", query)\
  .option("url", URL)\
  .option("user", USR)\
  .option("password", PWD)\
  .load()


tablename = "HOSP_DET"
query = f"SELECT * FROM {tablename}"

det_df = spark.read.format("jdbc")\
  .option("driver", drivername)\
  .option("query", query)\
  .option("url", URL)\
  .option("user", USR)\
  .option("password", PWD)\
  .load()

## 1 Criando um *target*

Vamos abordar este problema usando um modelo de classificação. Para isto vamos precisar de um variável target binária que separe os clientes entre "alto custo de aquisição" e "baixo custo de aquisição". Usaremos a probabilidade estimada de um cliente pertencer a um grupo ou à outro como o *score* que indica seu custo potencial de aquisição.